# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [ ]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

3. Разбейте текст формулировки задачи 2 на слова.

---
## Лабораторная работа 6.1

### Форматирование строк

6\.1\.1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [ ]:
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
recipes.head()

In [1]:
import pandas as pd

recipes_2 = pd.read_csv('recipes_sample.csv', delimiter=',')[['id','minutes']][7:12]
col = recipes_2.columns # Для вывода названий столбцов

max_id = max(map(str, recipes_2.id),key = len)  # Максимальная длина последовательности символов в столбце
m_id =  len(max_id)+4 # Определяем ширину столбцов

max_min = max(map(str, recipes_2.minutes),key = len)
m_min = len(max_min)+4

print(f'|{col[0]:^{m_id}}|{col[1]:^{m_min}}|')
print(f'|{(m_min+m_id+1)*"~"}|')

for k,v in zip(recipes_2.id,recipes_2.minutes):
    print(f'|{k:^{m_id}}|{v:^{m_min}}|')


|    id    |minutes|
|~~~~~~~~~~~~~~~~~~|
|  95926   |   5   |
|  453467  |  45   |
|  306168  |  40   |
|  50662   |  125  |
|  118843  |  20   |


6\.1\.2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [2]:
import requests
from bs4 import BeautifulSoup
    
with open('steps_sample.xml') as f:
    Book = BeautifulSoup(f, 'xml')
    
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')

dic_steps = dict();
for id_r in Book.recipes.find_all("recipe"):
    st = [steps.next for steps in id_r.find_all("step")]
    dic_steps[id_r.find("id").next] = st

In [3]:

    
def show_info(name, steps, author, minutes):
    string = f"{name}\n\n"
    for num, step in enumerate(steps):
        string += f"{num}. {step}\n"
    string+= f"-----------\nАвтор: {author}\nСреднее время приготовления: {minutes} минут"
    return string




name_r = recipes[recipes.id == 170895]['name'].values[0]
author = recipes[recipes.id == 170895]['contributor_id'].values[0]
minutes = recipes[recipes.id == 170895]['minutes'].values[0]
steps  =  dic_steps['170895']

print(show_info(name = name_r, author = author, minutes = minutes, steps= steps))

leeks and parsnips  sauteed or creamed

0. clean the leeks and discard the dark green portions
1. cut the leeks lengthwise then into one-inch pieces
2. melt the butter in a medium skillet , med
3. heat
4. add the garlic and fry 'til fragrant
5. add leeks and fry until the leeks are tender , about 6-minutes
6. meanwhile , peel and chunk the parsnips into one-inch pieces
7. place in a steaming basket and steam 'til they are as tender as you prefer
8. i like them fork-tender
9. drain parsnips and add to the skillet with the leeks
10. add salt and pepper
11. gently sautee together for 5-minutes
12. at this point you can serve it , or continue on and cream it:
13. in a jar with a screw top , add the half-n-half and arrowroot
14. shake 'til blended
15. turn heat to low under the leeks and parsnips
16. pour in the arrowroot mixture , stirring gently as you pour
17. if too thick , gradually add the water
18. let simmer for a couple of minutes
19. taste to adjust seasoning , probably an additio

In [ ]:
dic_steps

## Работа с регулярными выражениями

6\.1\.3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [4]:
import re

pattern = re.compile(r'(\d+ (?:hours|hour|minutes|minute))')
for step in dic_steps['25082']:
    if re.findall(pattern, step):
        print(re.findall(pattern, step))

    


['20 minutes']
['10 minutes']
['2 hours']
['10 minutes']
['20 minutes', '30 minutes']


6\.1\.4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [5]:
pattern = re.compile(r"(^this.*,[\s\S]but.*)") #Создание шаблона (".*" - Любой символ без ограничения повторений)
dt = recipes['description'].str.extract(pattern).dropna() 
#.str.exctract() - принимает шаблон от регулярок и извлекает значения по нему (принимает только группу(шаблон в скобочках))
for descriptions in dt[0][3:6]:
    print(f'\n{descriptions}\n')
print(f'Количество искомых рецептов: {dt[0].size}')


this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.


this is the simplest toffee recipe ever.  i love it because it only takes 4 ingredients that i always have around the house.  follow a few easy steps and voila--instant gratification!  this recipe makes great holiday gifts for the neighbors (they'll think you slaved over a hot stove all day, but the truth will be our little secret!).


this recipe is by don mauer, a syndicated food columnist. the technique used here makes for a fast yet tender pot roast. the 90 minutes refers to time in the oven, there is additional time needed for brining and cooking. do not cook vegetables with the pot roast - cook them separately or p

In [ ]:
dt

## Лабораторная работа 6.2

6\.2\.1\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
dic_steps['72367']


In [6]:
for step in dic_steps['72367']:
    print(re.sub(r' / ', '/', step))


mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\.2\.2\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [7]:
import nltk
nltk.download('punkt')
from nltk.tokenize.toktok import ToktokTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lekir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
from nltk.tokenize import sent_tokenize

In [9]:
words_s = []
for value in dic_steps.values():
    for step in value:
        toktok=ToktokTokenizer()
        words_s += toktok.tokenize(step)
words = [word for word in words_s if word.isalpha()]        

    


In [10]:
print(len(set(words)) ,'\n\n', set(words))

14956 

 {'unsafe', 'gloden', 'curds', 'done', 'diagongally', 'methold', 'clouding', 'pairs', 'wataer', 'parchement', 'soya', 'hit', 'judgement', 'saltier', 'proceeding', 'slithers', 'seems', 'maximum', 'drag', 'bridget', 'bla', 'galzed', 'examine', 'browling', 'mercy', 'decaf', 'expense', 'befroe', 'clumps', 'silverbeet', 'beeper', 'heate', 'cheerios', 'micorwave', 'plant', 'rerigerator', 'drizzed', 'brining', 'subsituted', 'mm', 'purposely', 'extraordinaire', 'boys', 'consistencythat', 'family', 'mouth', 'aux', 'beanies', 'stocks', 'foiled', 'inch', 'all', 'stirrying', 'amarenakirschen', 'whil', 'human', 'spoonful', 'dillweed', 'down', 'nudge', 'tablepoon', 'paremsan', 'arteries', 'indefinetely', 'korma', 'ghirardelli', 'ladles', 'crusts', 'goblet', 'salads', 'threatens', 'likeing', 'thorougha', 'love', 'mould', 'score', 'wiches', 'ballable', 'sifted', 'dean', 'halava', 'tb', 'allspice', 'stirr', 'tightening', 'alightly', 'discretion', 'vino', 'newcombe', 'parmesaon', 'blame', 'absor

6\.2\.3\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [115]:
# Создаем список описаний БЕЗ! NaN
des_lst = recipes['description'].dropna().to_list()

# Разбиваем на предложения (sent_lst - список списков предложений) 
sent_lst = [nltk.sent_tokenize(description) for description in des_lst]

# Сортируем по кол-ву предложений в описании
sentence_lst = sorted(sent_lst, key=len)

top_5 = sentence_lst[-5:] 
min_top_5 = list(lst[0] for lst in top_5) # Первые предложения описаний

# Если первое предложение из top_5 есть в описании des_lst - выводим это описание
desc_top_5 = list(full_des for full_des in des_lst for sent in min_top_5 if sent in full_des ) 

recipes[recipes.description.isin(desc_top_5)]

# Теряются символы после точек, например, "tree will grow.   Etc." будет ['tree will grow.','Etc.'].
# Не делать с помощью " ".join()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0


6\.2\.4\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [11]:
from nltk import pos_tag
from nltk import pos_tag, word_tokenize

w1 = 'I  omitted the raspberries and added strawberries instead'
w2 = recipes[recipes['id'] == 241106]['name'].values[0]

def pos_word(word):
    for i in pos_tag(word_tokenize(word)):
        print(f'{i[1]: ^{len(i[0])+len(i[1])}}', end = '')
    print('')
    for i in pos_tag(word_tokenize(word)):
        print(f'{i[0]: ^{len(i[0])+len(i[1])}}', end = '')
    print('\n')

pos_word(w1)
pos_word(w2)

PRP    VBD     DT       NNS       CC    VBD         NNS         RB    
 I   omitted   the  raspberries   and  added   strawberries   instead 

    JJ       NNS     IN      NNS       VBP     JJ    CC    JJ      NNS   
 eggplant  steaks   with  chickpeas   feta   cheese  and  black  olives  

